<div style="background: black; padding: 10px 250px"><img src="https://www.veldikompetens.se/wp-content/themes/consid/static/icons/VeldiKompetens_Logo_Web_Negative.svg" title="Veldi kompetens" /></div>

<hr><h1><center>Assignment 4 - Simulating a Pandemic</center></h1>

<h3>Instructions </h3>

<p>In this weeks assginment you will be given the initial code for a simulation program which you will improve upon to make more realistic, the case we will be working with is a simulation of a pandemic 

It is heavily recommended that you first go through the code slowly with print statements and maybe also make your own comments to fully understand how the code works. Some key concepts here are: Constructors (the __init__ methods), classes, Object Oriented Programming and methods.

Although this week definitely is a bit more heavy on the coding side, a part of this course is still to also learn Python and not just work with libraries such as numpy and pandas (although we use them). It is also important to know how to use Python to recreate realisitic scenarios. By learning this skill you will become better at understanding real life data and how dynamics in a system work as well as become a better Python programmer.

Best of luck!</p>

<h3> 1. Setup </h3>
<p> For this assignment we will be using mostly just regular Python but also a bit of numpy for efficient storage. Random will be used for generating probabilities </p>

In [436]:
import numpy as np
import random

<h3> 2. Modelling a Pandemic</h3>
<p> NOTE: The scenario is made to be the most simple format reasonable. There are a lot of realistic factors not taken into consideration as that is partially the point of the assignment, but also that fully simulating reality is almost impossible. It is also important to know that each time you run the simulation the results will differ as parts of the simulation such as initial sick population is randomized.</p>
<h4>2.1 The class Person </h4>
First of we have the class Person which is an inhabitant in the village. A person initially has attributes such as:

<ul>
    <li>A fixed probability of getting: Sick, Well and pass away</li>
    <li>Three different states; Either sick, dead or recovered</li>
    <li>A method init_sick_or_not which is run when a person object is created. When run it sets the person as initially either sick or not using a randomly generated probability, if this probability is less than or equal to the init_sick_prob the person is set to sick. You can think of it this way: The 0.1 kind of symbolizes 10% which means that we should approximately get that 10% of the population is sick. We randomize a number to get a varying amount of initiallty sick individuals so sometimes it might be that 11% of the population is sick, sometimes 9%. This is to add versatility to our simulation and make it more randomized. You will see this approach being replicated a lot in this assignment </li>
    <li>A method called day_passes which is ran for every day the simulation is running. A person might then get sick, recover or pass away depending on their current status </li>
</ul>

In [437]:
class Person:
    def __init__(self):
        self.recover_prob = 0.2
        self.die_prob = 0.05
        self.init_sick_prob = 0.1
        self.recovered = False
        self.dead = False
        self.sick = False
        
        #Additional variables for task1
        self.infect_others_prob = 0.05
        #self.average_meetups = not considered as probability but as in absolute number of meetups
        self.average_meetups = 10
        #self.days_sick to track how many days an individual has been sick.
        self.days_sick = 0
        
        
        self.init_sick_or_not()
        
    def init_sick_or_not(self):
        """When created each person starts as either sick or healthy"""
        prob = random.random()
        if prob <= self.init_sick_prob:
            # This should yield that about 10% of the people created are sick, the rest healthy
            self.sick = True
        else:
            self.sick = False
    
    def day_passes(self, population, init_scenario):
        """This method describes what happends to each person each day"""
                      
        # Will person recover?
        prob = random.random()
        
        #print(f"The current value of self.sick is {self.sick}.\n")
        #print(f"The current value of init_scenario is {init_scenario}.\n")
        #Additional code for the exercise
        if self.sick == True:
            #print("The citizen is sick")
            self.days_sick +=1
            print(f"The number of sick days is:", self.days_sick)
            if init_scenario == False:
                #print("we are in init__scenario equal to false, i.e. at least 2nd day of pandemic")
                self.infect_others(population)
        
        
        #Initial code code for the exercise
        if prob <= self.recover_prob and self.sick == True:
            # Person has rehabilitated and is now healthy!
            self.sick = False
            self.recovered = True 
        
        # If person is still sick they might die
        prob = random.random()
        if prob <= self.die_prob and self.sick == True:
            self.dead = True
            self.sick = False
            
    def infect_others(self, population):
        #print(population.size)
        person_encounters = random.sample(range(population.size), self.average_meetups)
        #print(f"The value of person_encounters is:",person_encounters)
        for person_id in person_encounters:
            #print(f"We are checking the person: {person_id}.")
            prob = random.random()
            #print(f"We are checking the dead status of the person {person_id}: {population[person_id].dead}.")
            #print(f"We are checking the recovery status of the person {person_id}: {population[person_id].recovered}.")
            
            if prob <= self.infect_others_prob and ((population[person_id].dead == False) and (population[person_id].recovered == False)):
                #print("======================================================================")
                #print(f"Sick status of the person {person_id} before being updated: {population[person_id].sick}.")
                #print("======================================================================")
                population[person_id].sick = True
                #print("Another individual has been infected.")
                #print(f"The individual probability number is {prob} less than {self.infect_others_prob}.")
                #print(f"We are checking the dead status of the person {person_id}: {population[person_id].dead}.")
                #print(f"We are checking the recovery status of the person {person_id}: {population[person_id].recovered}.")
                #print(f"Sick status of the person {person_id} after being updated: {population[person_id].sick}.")
                #print("\n")
            
            #print("\n")
            
        #print("\n")
        
        
        

<h4>2.2 The class Village </h4>
The village contains the population (persons) and is where the population lives. The village class acts as kind of the steering wheel, initiating the simulation and then running it until everyone in the population has either recovered or the disease has become fully extinct. The class contains:

<ul>
    <li>The population which is a numpy array (using np.append is quite inefficient so the most efficient way of filling an array of pre-known size is creating an empty one and then looping over the size and setting each index to a value </li>
    <li>A method for generating the population (generate_inhabitants)</li>
    <li>A method for counting the status of people in the village and calling each persons day passes function which in turn decides if the person recovers, dies or remains sick (advance_days) </li>
    <li>A method which controls the simulation and progress the days forward. This is probably the most important method as it controls the whole program (start_simulation). You will not be modifying this method a lot though, most work will be done in advance_days and the person class </li>
</ul>

In [438]:
class Village:
    def __init__(self, init_population_size):
        #print("\n\n")
        #print(f"The type of Person is:",type(Person))
        #print(f"Person is:",(Person))
        
        self.population = np.empty(init_population_size, Person)
        
        #print(f"The type of self.population is:",type(self.population))
        #print(f"The dimension of self.population is:",(self.population.ndim))
        #print(f"The shape of self.population is:",(self.population.shape))
        #print(f"The size of self.population is:",(self.population.size))
        #print("\n\n")
        self.generate_inhabitants()
    
    def generate_inhabitants(self):
        """Generates the population"""
        #print("\n\n")
        for i in range(self.population.size):
            self.population[i] = Person()
            #print(f"What we storage in self.population[i] is actually:", self.population[i])
            #print(f"The type of self.population[i] is actually:", type(self.population[i]))
            #print(f"The sickness state of the individual {i} is {self.population[i].sick}")
            #print(f"The death state of the individual {i} is {self.population[i].dead}")
            #print(f"The recovery state of the individual {i} is {self.population[i].recovered}")
            #print("\n")
            
        #print("\n\n")
        
    def advance_days(self, init_scenario = False):
        """Counts the status of the citizens in the community"""
        people_sick = 0
        people_recovered = 0
        people_dead = 0
        #My own variables to understand how the code works.
        counter_for_loop = 0
        
        for person in self.population:
            if person.sick == True:
                people_sick += 1
            elif person.recovered == True:
                people_recovered += 1
            elif person.dead == True:
                people_dead += 1
            
            #print(f"The type of person is:", type(person))
            #print(f"The sickness state of the individual is {person.sick}")
            #print(f"The type of init_scenario is:",type(init_scenario))
            #print(f"The current value of init_scenario is:",(init_scenario))
            #print(f"The inhabitant of the village we are checking is:",(counter_for))
            counter_for_loop += 1
            # init_scenario is true for the first day of the pandemic and false the rest days.
            
            person.day_passes(self.population, init_scenario)
            
            #print("\n")

        return people_sick, people_recovered, people_dead
    
    def start_simulation(self):
        """This function controls the simulation and what happends in a day"""
        current_day = 0
        days_sick_list = []
        
        people_sick, people_recovered, people_dead = self.advance_days(init_scenario = True)
        while people_sick != 0:
                                    
            print(f"By day {current_day}, {people_sick} people are sick, {people_dead} are dead and {people_recovered} have recovered")
            current_day += 1
            people_sick, people_recovered, people_dead = self.advance_days()
            
            
            for i in range(self.population.size):
                #print(f"We are accessing the individual: {i}")
                print(f"The number of sick days, for the person {i+1}, are: {self.population[i].days_sick}\n")
                #days_sick_list.append(self.population[i].days_sick)
            #print(days_sick_list)
            
        print(days_sick_list)
        people_unaffected = self.population.size - (people_sick + people_recovered + people_dead)
        print(f"\nBy day {current_day} {people_sick} people are sick, {people_dead} are dead and {people_recovered} have recovered. {people_unaffected} people were never in contact with the virus")
        print("The village has recovered and the virus is eliminated!")

        
        

<h4>2.3 The main function </h4>
<p>The main function is from where all the code is ran and initializes the simulation with a start population size and creates an instance of the village class. If you want to rerun the whole simulation you just need to rerun the function below as it runs everything else.</p>

In [440]:
def main():
    pop_size = 100
    village = Village(pop_size)
    village.start_simulation()

main()

The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 1
By day 0, 15 people are sick, 0 are dead and 0 have recovered
The number of sick days is: 2
The number of sick days is: 2
The number of sick days is: 1
The number of sick days is: 2
The number of sick days is: 2
The number of sick days is: 2
The number of sick days is: 2
The number of sick days is: 2
The number of sick days is: 2
The number of sick days is: 2
The number of sick days is: 1
The number of sick days is: 2
The number of sick days is: 2
The number of sick days is: 1
The number of sick days is: 1
The number of sick days is: 2
The numb

<h3>3. The assignment </h3>
<p>Hopefully you are now familiar with the model above and understand how it works. Sadly for the upcoming part jupyter notebook does not really allow classes to be modified so easily in separate cells. So for this assignment you will be handed tasks below with references to where you need to modify the code to achieve the desired results. Hints will be given but it is expected that your understand the model above to properly know where to edit Toy around with it and add printstatements if you would like, it is very very important that you fully understand the mode before proceeding. Best of luck! </p>

<h4>3.1 Implementing transmission (i.e the virus spreading) </h4>
<p>In the model above we only have a set initial amount of infected which then go through the process of either recovering or succumbing to the virus. We will now implement a way for the virus to spread.</p>

<p>As it happends, the particular virus we are dealing with has 5% chance of spreading if a sick individual gets in contact with a person that is susceptible (Kind of corresponds to that if you meet 20 people, one will get infected on average). A susceptible person is someone that is not dead, is not already sick  has not recovered from the virus, which we say would make you immune. On average each person in the village meets 10 people a day. </p>

<h5>Instructions </h5>
<p>Do like this (All of the following is done in the Person class): </p>

<ul>
    <li style="padding-bottom: 16px;">In the constructor for the Person class add two variables. self.infect_others_prob which you set to the chance of spread described above and another variable self.average_meetups which is set to the average amount of people met each day </li>
    <li style="padding-bottom: 16px;">Create a method infect_others with the parameters self and population. In the method create a variable person_encounters which should sample as many as self.average_meetups people from the population. To sample indexes from the population array you can use random.sample (Google it!). I recommend that you sample indexes and not the people themselves, a way of doing this is something like this: random.sample(range(size of population), how many you want). </li>
    <li style="padding-bottom: 16px;">Having used random.sample your person_encounters list should now be a list of 10 indexes of people from the population array. Write a forloop over person_encounters with a loop variable person_id where in each loop you generate a probability like we do in day_passes and then check if the generated probability is less than or equal to self.infect_prob, also since this represents the way of infecting another individual include in the if-statement a check for whether the person is not dead nor has already recovered from the virus (Note: since we are looping over indexes to check the status of an individual you need to do something like population[person_id] to get the specific person). In total your if-statement should thus include 3 criteria at the moment; that the prob variable is less than self.infect_prob, that the person about to be potentially infected is not dead and has not previously recovered from the virus. </li>
    <li style="padding-bottom: 16px;">if all of these conditions are met set the persons condition to sick. if done correctly it should look something like this population[person_id].sick = True. Where person_id is the loop variable. </li> 
    <li style="padding-bottom: 16px;">Lastly we need to add a call to this function from day_passes. At the beginning of the day_passes method first write an if statements that checks if the person we are currently working on is sick (currently working on refers to self). Then, inside of that if statement nest another if statement which checks if self.init_scenario == False (we do not want spread to happen on day 0). If the criteria are met call the infect_others method inside the second if statement </li>
</ul>

<h5>A short discussion </h5>
<p>Something that might strike you as odd is that when we modified that status of a person in the infect_others method the change we made to the population variable also affected the variable in the villager class without having to do a return and refresh the value of it, this is sort of a beauty and downside of numpy arrays. Recommended reading on the basics of it on numpy docs amount arrays. If you do not think its wierd then no worries:) </p>

<p>As the results may differ quite a lot with each run its impossible to say exactly what you will get but it should be something like this once you run the main method. </p>


<h5>Somewhat or similarly expected output when you run the main method: </h5>
<pre>By day 0 108 people are sick, 0 are dead and 0 has recovered
By day 1 99 people are sick, 4 are dead and 23 has recovered
By day 2 111 people are sick, 9 are dead and 43 has recovered
By day 3 129 people are sick, 13 are dead and 62 has recovered
By day 4 155 people are sick, 20 are dead and 86 has recovered
By day 5 183 people are sick, 22 are dead and 120 has recovered
By day 6 200 people are sick, 27 are dead and 152 has recovered
By day 7 207 people are sick, 37 are dead and 194 has recovered
By day 8 213 people are sick, 43 are dead and 236 has recovered
By day 9 193 people are sick, 57 are dead and 282 has recovered
By day 10 177 people are sick, 62 are dead and 327 has recovered
By day 11 171 people are sick, 70 are dead and 364 has recovered
By day 12 152 people are sick, 81 are dead and 403 has recovered
By day 13 143 people are sick, 88 are dead and 429 has recovered
By day 14 138 people are sick, 94 are dead and 456 has recovered
By day 15 123 people are sick, 102 are dead and 486 has recovered
By day 16 108 people are sick, 107 are dead and 509 has recovered
By day 17 105 people are sick, 108 are dead and 525 has recovered
By day 18 95 people are sick, 110 are dead and 545 has recovered
By day 19 85 people are sick, 114 are dead and 568 has recovered
By day 20 76 people are sick, 119 are dead and 581 has recovered
By day 21 67 people are sick, 120 are dead and 597 has recovered
By day 22 60 people are sick, 121 are dead and 612 has recovered
By day 23 55 people are sick, 123 are dead and 621 has recovered
By day 24 53 people are sick, 125 are dead and 629 has recovered
By day 25 48 people are sick, 126 are dead and 637 has recovered
By day 26 37 people are sick, 129 are dead and 648 has recovered
By day 27 30 people are sick, 129 are dead and 658 has recovered
By day 28 24 people are sick, 130 are dead and 664 has recovered
By day 29 20 people are sick, 130 are dead and 670 has recovered
By day 30 22 people are sick, 132 are dead and 670 has recovered
By day 31 15 people are sick, 133 are dead and 677 has recovered
By day 32 16 people are sick, 134 are dead and 677 has recovered
By day 33 15 people are sick, 134 are dead and 680 has recovered
By day 34 13 people are sick, 134 are dead and 683 has recovered
By day 35 14 people are sick, 134 are dead and 685 has recovered
By day 36 8 people are sick, 135 are dead and 691 has recovered
By day 37 6 people are sick, 135 are dead and 694 has recovered
By day 38 6 people are sick, 135 are dead and 695 has recovered
By day 39 5 people are sick, 135 are dead and 696 has recovered
By day 40 5 people are sick, 135 are dead and 696 has recovered
By day 41 4 people are sick, 135 are dead and 697 has recovered
By day 42 4 people are sick, 135 are dead and 697 has recovered
By day 43 3 people are sick, 135 are dead and 698 has recovered
By day 44 1 people are sick, 136 are dead and 699 has recovered
By day 45 1 people are sick, 136 are dead and 699 has recovered

By day 46 0 people are sick, 136 are dead and 700 has recovered. 164 people were never in contact with the virus
The village has recovered and the virus is eliminated!
</pre>

<h4>3.2 Implementing a days_infected tracker</h4>
<p>Now we will implement something that might be of interest, namely a way of documenting how long an individual has been sick. By doing this we can then find the minimum and maximum days spent sick by people in the village which is interesting and good data to have.</p> 

<h5>Instructions </h5>
<p>Do like this (All of the following is done in the Person class): </p>

<ul>
    <li style="padding-bottom: 16px;">In the constructor for the Person class add a variable self.days_sick which you initiallty set to 0 </li>
    <li style="padding-bottom: 16px;">In the first if-statement we created in day_passes that checked if the individual was sick, increment the self.days_sick variable by one if true. Note: Put it before the second if statement, NOT after or inside the second if statement </li>


</ul>

<p>Next up lets add the functionality to the Village class (All of the following is done in the Village class) </p>

<ul>
    <li style="padding-bottom: 16px;">After the while statement in start_simulation do a forloop that fills a list of the same size as our population array but with each persons .days_sick variable. Create the array wherever you want and how you want but in the end the list should contain each persons .days_sick variable value. The best way is using list comprehension.  </li>
    <li style="padding-bottom: 16px;">After the last print statement i.e "The village has recovered..." create a new print statement where you print the max value of the list you just created. You can use max(your-list). Below is my output for a run where I only include the final parts of the print statement </li>
</ul>

<h5>Somewhat expected output: </h5>
<pre>
By day 46 0 people are sick, 151 are dead and 688 has recovered. 161 people were never in contact with the virus
The village has recovered and the virus is eliminated!
The longest time an individual was sick is:  35 days
</pre>




<h4>3.3 Implementing vaccination </h4>
<p>It turns out the community has experienced viruses before and once it becomes a known problem in the community villagers start vaccinating against it. Each person only needs one dose to reach full immunity, however worth of note is that being fully vaccinated does not count as having recovered from the disease. Awareness about the virus usually become apparent once it reaches about 15-25% of the population (we will thus go with the idea that once 20% of the population have been/become sick vaccination starts). For simplicity sake we say that 4% of the population which are neither sick, dead nor already vaccinated each day gets vaccinated once the process is started, meaning that the doctors cant really tell if someone has already recovered but they obviously can tell if someone already has become vaccinated. Lets save some lives! </p>
<h5>Instructions</h5>

<p>Do like this (All of the following is done in the Person class): </p>

<ul>
    <li style="padding-bottom: 16px;">In the constructor create a new variable self.vaccinated and set it to False</li>
    <li style="padding-bottom: 16px;">Modify the if statement in infect_others to include a check that the person about to be infected is not already vaccinated. There should now be 4 conditions in the if statement in total. That is all we need to do for the Person class, lets proceed to the village class!</li>

</ul>

<p>Next up the village class (All of the following is done in the Village class) </p>

<ul>
    <li style="padding-bottom: 16px;">Create three variables in the constructor. self.init_vaccination which you set to 0.2*init_population_size which corresponds to when vaccination will start, a second variable called self.vaccination_started which you set to False and the third being daily_vaccination_threshold which is set to 0.04*init_population size</li>
    <li style="padding-bottom: 16px;">In the advance_days method replicate how we did with the people_recovered, people_sick and people_dead variables but with a new variable called people_vaccinated. The people_vaccinated variable should increment if person.vaccinated == True. Add the people_vaccinated variable to the return statement at the end of the method. Remeber to add the variable in the start_simulation method just as with the others! This allows us to track how many are vaccinated if we wish.</li>
    <li style="padding-bottom: 16px;">Create a new method called vaccinate_population and for now just type pass inside of it as a placerholder. After the forloop in advance_days write an if statement checking if people_sick >= self.init_vaccination and self.vaccination_started == False, if both these conditions are met change the value of self.vaccination_started to True and add a print statement saying something alike "At the end of the day {amount of people sick} have become sick and a vaccination process has been initiated!". Just after this if statement add another if statement where you check if self.vaccination_started == True, if so call the vaccinate_population method. Make sure the print in the first if statement only runs once, otherwise you have some bug in your code.</li>
    <li style="padding-bottom: 16px;">In the vaccinate_population method first create a variable called people_vaccinated_today which you set to zero. Then write a forloop that goes through the population with a loop variable person which represents each individual like we do in advance days. In the forloop write an if-statement checking whether people_vaccinated_today is equal to the daily_vaccination_threshold, if so break the forloop (using the keyword break) since we have reached maximum amount of people being vaccinated daily. Else if (write an elif statement) check whether that the person we are currently examining from the population is susceptible (i.e is not sick, is not dead and is not already vaccinated). If the conditions are met set that persons vaccination status to true and then increment the people_vaccinated_today variable by one. </li>

</ul>

<p>You are now good to go and have implemented vaccination into the village! Awesome! Rerun the code a couple of times and you should see that sometimes the vaccinations does not start and sometimes it does. Whenever vaccination is introduced a lot fewer people become sick (for me it is usually almost about half the amount of people). Here is the summary output of two of my runs, one where no vaccination was started and one where it was (remeber you will never get the exact same outputs but you should get something somewhat similar)</p>

<h6>Somewhat expected output no vaccination process</h6>
<pre>
By day 54 0 people are sick, 149 are dead and 654 has recovered. 197 people were never in contact with the virus
The village has recovered and the virus is eliminated!
The longest time an individual was sick is:  23 days
</pre>
<h6>Somewhat expected output when vaccination occurs</h6>
<pre>
By day 37 0 people are sick, 140 are dead and 588 has recovered. 272 people were never in contact with the virus
The village has recovered and the virus is eliminated!
The longest time an individual was sick is:  27 days
</pre>

<h3> 4. Closing remarks </h3>
<p>Well done! You are now done with this weeks sort of heavy assignment! There was a lot to do but we have had a lot of fun along the way and hopefully you have learned to navigate the code and understand how to implement realistic things into our simulation. There are tons of more things to do to make it realistic, but this is a good start. In next week we will reuse this code to combine it with pandas and generate a dataframe which we will then convert to a storeable csv file. See you next week and once again: WELL DONE!!</p>

<h2>Remember to push your assignment to github and add a link to your repo in the handin for the assignment on itslearning:)</h2>